In [0]:
-- observe the data sample from fact table
select * from silver_sb.test2.stg_fact_transactions limit 20

order_id,line_id,type,dt,pos_site_id,sku_id,fscldt_id,price_substate_id,sales_units,sales_dollars,discount_dollars,original_order_id,original_line_id
164087401,2,Sale,2016-01-31T06:17:01.000Z,CATMAIN,2668940801,20160131,FP,1,58.95,0.0,null,null
164087409,4,Sale,2016-01-31T06:17:25.000Z,CATMAIN,2920920601,20160131,FP,1,49.95,0.0,null,null
164087440,2,Sale,2016-01-31T06:19:28.000Z,INETMAIN,0695690000,20160131,FP,2,37.9,0.0,null,null
164087469,2,Sale,2016-01-31T07:03:00.000Z,CATMAIN,0505490000,20160131,FP,1,45.0,0.0,null,null
164087472,2,Sale,2016-01-31T07:07:36.000Z,CATMAIN,2872180801,20160131,FP,1,98.95,0.0,null,null
164087476,1,Sale,2016-01-31T07:16:49.000Z,CATMAIN,4840520704,20160131,FP,2,139.9,0.0,null,null
164087482,1,Sale,2016-01-31T07:28:10.000Z,CATMAIN,0403081000,20160131,FP,1,22.95,0.0,null,null
164087501,2,Sale,2016-01-31T08:26:32.000Z,CATMAIN,0700002000,20160131,FP,1,24.95,0.0,null,null
164087518,1,Sale,2016-01-31T11:10:29.000Z,CATMAIN,2598481801,20160131,FP,1,59.95,0.0,null,null
164087571,1,Sale,2016-01-31T13:38:18.000Z,CATMAIN,2815121701,20160131,FP,1,118.95,0.0,null,null


2c) Created refined table with all data available

In [0]:
-- considered data that have matched week_ids from dimension table
-- left join with week_id as null filter can also be used here instead of inner join

create or replace table gold_sb.test1.mview_weekly_sales as
select pos_site_id, sku_id, fsclwk_id, price_substate_id, type, sum(sales_units) as wk_sales_units, sum(sales_dollars) as wk_sales_dollars, sum(discount_dollars) as wk_discount_dollars from 
(select pos_site_id, sku_id, ft.fscldt_id, hc.fsclwk_id, price_substate_id, type, sales_units, sales_dollars, discount_dollars from 
silver_sb.test2.stg_fact_transactions ft  --4493723; selecting required columns from fact table
inner join silver_sb.test2.stg_hier_clnd hc on ft.fscldt_id = hc.fscldt_id  --inner join to get the corresponding week_id from dim table
)
group by pos_site_id, sku_id, fsclwk_id, price_substate_id, type --grouping to get the resultant sums
order by fsclwk_id desc

num_affected_rows,num_inserted_rows


In [0]:
select count(*) from gold_sb.test1.mview_weekly_sales 

count(*)
887849


2d) Incremental process acheived by applying filter on fscldt_id

In [0]:
-- considered data before 20200415 that leads to 115 weeks data considering only matched week_ids from dimension table
-- left join with week_id as null filter can also be used here instead of inner join

create or replace table gold_sb.test1.mview_weekly_sales as
select pos_site_id, sku_id, fsclwk_id, price_substate_id, type, sum(sales_units) as wk_sales_units, sum(sales_dollars) as wk_sales_dollars, sum(discount_dollars) as wk_discount_dollars from 
(select pos_site_id, sku_id, ft.fscldt_id, hc.fsclwk_id, price_substate_id, type, sales_units, sales_dollars, discount_dollars from 
silver_sb.test2.stg_fact_transactions ft  --4493723; selecting required columns from fact table
inner join silver_sb.test2.stg_hier_clnd hc on ft.fscldt_id = hc.fscldt_id  --inner join to get the corresponding week_id from dim table
where ft.fscldt_id < '20200415') --4493723; 115 weeks
group by pos_site_id, sku_id, fsclwk_id, price_substate_id, type --grouping to get the resultant sums
order by fsclwk_id desc

num_affected_rows,num_inserted_rows


In [0]:
select count(*) from gold_sb.test1.mview_weekly_sales --886817 initial count remaining should be there in incremental load

count(1)
886817


In [0]:
-- considered data above 20200415 that leads to 2 weeks data considering only matched week_ids from dimension table
-- The following can be executed daily by changing the fscldt_id value in the first 2 ctes

with fact_trans as (
  select pos_site_id, sku_id, fscldt_id, price_substate_id, type, sales_units, sales_dollars, discount_dollars from 
silver_sb.test2.stg_fact_transactions where fscldt_id >= '20200415'
), --9385 fetching required data by giving the fscldt_id range
hier_weeks as (
  select fscldt_id, fsclwk_id from silver_sb.test2.stg_hier_clnd where fscldt_id >= '20200415'
), --1019 fetching fsclwk_id from dim table for the required range
new_trans_data as(
  select fact_trans.*, hier_weeks.fsclwk_id from fact_trans inner join hier_weeks on fact_trans.fscldt_id = hier_weeks.fscldt_id 
), --9385 joining fact table with dim table to get the week_id
new_week_sales as(
  select pos_site_id, sku_id, fsclwk_id, price_substate_id, type, sum(sales_units) as wk_sales_units, sum(sales_dollars) as wk_sales_dollars, sum(discount_dollars) as wk_discount_dollars from new_trans_data 
  where fsclwk_id is not null 
  group by pos_site_id, sku_id, fsclwk_id, price_substate_id, type
), -- summation on newly added data
previous_partial_data as(
  select * from gold_sb.test1.mview_weekly_sales where fsclwk_id = (select min(fsclwk_id) from new_trans_data)
), -- fetching the lastest weeks data from the existing table to avoid any resultant mistake
combined_sales as(
  select * from new_week_sales where fsclwk_id is not null
  union all
  select * from previous_partial_data
), -- combining the new data with the latest week of existing data for handling partial week case
final_data as(
  select pos_site_id, sku_id, fsclwk_id, price_substate_id, type, sum(wk_sales_units) as wk_sales_units, sum(wk_sales_dollars) as wk_sales_dollars, sum(wk_discount_dollars) as wk_discount_dollars from combined_sales 
  group by pos_site_id, sku_id, fsclwk_id, price_substate_id, type
) -- grouping the data to get the final result

--  performing incremental using the merge statement as follows
merge into gold_sb.test1.mview_weekly_sales mws using final_data fd ON
mws.pos_site_id = fd.pos_site_id and mws.sku_id = fd.sku_id and mws.fsclwk_id = fd.fsclwk_id and mws.price_substate_id = fd.price_substate_id and mws.type = fd.type
WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT *

-- num_affected_rows	num_updated_rows	num_deleted_rows	num_inserted_rows
-- 1637	605	0	1032

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1637,605,0,1032


In [0]:
select count(*) from gold_sb.test1.mview_weekly_sales --886817+1032 = 887849 count after incremental which is equal to the 2c count

count(1)
887849
